## Raw data from Lars Santema

There is a device that measured oxygen concentration and first we have to extract the necessary information out of the csv files

In [1]:
import csv
import pyenzyme as pe

In [65]:
def export_raw_data(csv_path):
    with open(csv_path) as csvfile:
        csv_file = csv.reader(csvfile, delimiter=',')
        check = 0
        time = []
        oxygen_conc = []
        oxygen_rate = []
        for row in csv_file:
            if not row:
                check = 0
            elif row[0] == "Time":
                check = 1
            elif check == 1:
                time.append(float(row[0]))
                oxygen_conc.append(float(row[1]))
                if row[2] == "INVALID":
                    row[2] = 0
                oxygen_rate.append(float(row[2]))
        return time, oxygen_conc, oxygen_rate

In [66]:
path = './0.5mM NAG + 0.05mM NAGOX 1.csv'
time, oxygen_conc, oxygen_rate = export_raw_data(path)

In [100]:
def create_EnzymeML(name, time, oxygen_conc, oxygen_rate, init_conc_glucosamine, init_conc_e):
    enzmldoc = pe.EnzymeMLDocument(name = name)

    # Create a vessel and add it to the document
    vessel = pe.Vessel(name="Oxygraph chamber", volume=1.0, unit="ml")
    vessel_id = enzmldoc.addVessel(vessel)

    # Set up reactants and proteins from databases
    enzyme = pe.Protein(name="N-acetylglucosamine oxidase (NagOx)", vessel_id=vessel_id)
    enzyme_id = enzmldoc.addProtein(enzyme)

    oxygen = pe.Reactant(name="Oxygen", vessel_id=vessel_id, unit="nmole / l")
    oxygen_id = enzmldoc.addReactant(oxygen)

    glucosamine = pe.Reactant(name="N-acetyl-D-glucosamine", vessel_id=vessel_id, unit="mmole / l")
    glucosamine_id = enzmldoc.addReactant(glucosamine)

    h2o2 = pe.Reactant(name="Hydrogen peroxide", vessel_id=vessel_id)
    h2o2_id = enzmldoc.addReactant(h2o2)

    lactone = pe.Reactant(name="N-acetyl-D-glucosaminolactone", vessel_id=vessel_id)
    lactone_id = enzmldoc.addReactant(lactone)

    # Build the reaction
    
    reaction = pe.EnzymeReaction.fromEquation(
        equation="Oxygen + N-acetyl-D-glucosamine -> N-acetyl-D-glucosaminolactone + Hydrogen peroxide",
        ph=7.5,
        temperature=25,
        temperature_unit="C",
        modifiers=[enzyme_id],
        name="N-acetylglucosamine conversion",
        enzmldoc=enzmldoc
    )

    reaction.addEduct(species_id=oxygen_id, stoichiometry=1.0, enzmldoc=enzmldoc)
    reaction.addEduct(species_id=glucosamine_id, stoichiometry=1.0, enzmldoc=enzmldoc)
    reaction.addProduct(species_id=h2o2_id, stoichiometry=1.0, enzmldoc=enzmldoc)
    reaction.addProduct(species_id=lactone_id, stoichiometry=1.0, enzmldoc=enzmldoc)

    # ... and add it to the document
    reaction_id = enzmldoc.addReaction(reaction)

    # First Measurement
    measurement_1 = pe.Measurement(name="Measurement", global_time_unit="mins")

    # Add each entity that will be measured
    measurement_1.addData(reactant_id=oxygen_id, init_conc=oxygen_conc[0], unit="nmole / l")
    measurement_1.addData(reactant_id=glucosamine_id, init_conc=init_conc_glucosamine, unit="mmole / l")
    measurement_1.addData(reactant_id=h2o2_id, unit="mmole / l")
    measurement_1.addData(reactant_id=lactone_id, unit="mmole / l")
    measurement_1.addData(protein_id=enzyme_id, init_conc=init_conc_e, unit="mmole / l")

    # Add it to the EnzymeML document
    meas_1_id = enzmldoc.addMeasurement(measurement_1)

    # Adding raw data 
    repl_oxygen_conc = pe.Replicate(
        id="oxygen_conc",
        species_id=oxygen_id,
        data_unit="nmole / l",
        time_unit="s",
        time=time,
        data=oxygen_conc
    )

    # Add it to the first measurement
    meas = enzmldoc.getMeasurement(meas_1_id)
    meas.addReplicates([repl_oxygen_conc], enzmldoc=enzmldoc)

    # Adding the Model

    model_generator = pe.KineticModel.createGenerator(
        name= "irreversible Michaelis-Menten",
        equation= "vmax * substrate / (Km + substrate)",
        #substrate=substrate_id,
        vmax={"unit": "mole / l * s"}, 
        Km={"unit": "mmole / l"}
    )

    model = model_generator(substrate=oxygen_id)

    reaction.model = model

    # To an OMEX archive
    enzmldoc.toFile(".", name=name)

In [101]:
# name, time, oxygen_conc, oxygen_rate, init_conc_glucosamine, init_conc_e
create_EnzymeML("0.5mM_NAG_0.05mM_NAGOX", time, oxygen_conc, oxygen_rate, 0.5, 0.05)


Archive was written to ./0.5mM_NAG_0.05mM_NAGOX.omex

